# Análisis y Diseño para el Desafío II: UdeATunes
**Informática II - Semestre 2025-2**

**Hecho por:** *Oscar David Gutierrez Hernandez*

**Presentado a:** *Augusto Enrique Salazar Jimenez*

**Fecha:** *15 de Octubre de 2025* 

Este documento presenta el análisis del problema planteado en el Desafío II y el diseño preliminar de la solución utilizando Programación Orientada a Objetos (POO) en C++. Se enfoca en los requisitos para la entrega parcial: análisis del problema, consideraciones de la solución propuesta, diagrama de clases UML y descripción de lógica. El desarrollo se basa en el enunciado proporcionado, priorizando eficiencia (sin STL, memoria dinámica propia) y conceptos POO (abstracción, encapsulación, relaciones, sobrecarga, plantillas).

## 1. Análisis del Problema

### ¿De qué se trata este desafío?

Bueno, básicamente tengo que crear una versión simplificada de Spotify, pero para la universidad. El programa se llama "UdeATunes" y la idea es simular todas las cosas que pasan en un servicio de música en streaming real: usuarios que se registran, artistas que suben música, álbumes, canciones, listas de favoritos, y hasta anuncios publicitarios.

**¿Qué tipos de usuarios tendré?**
- **Usuarios estándar**: Son los que tienen acceso básico. Pueden escuchar música, pero solo a 128kbps (calidad normal) y les aparecen anuncios cada dos canciones. 
- **Usuarios premium**: Estos son los que pagan. Tienen acceso a música en alta calidad (320kbps), no les salen anuncios, y pueden crear listas de favoritos con hasta 10.000 canciones. También pueden seguir las listas de otros usuarios premium.

**¿Qué entidades voy a manejar?**
- **Artistas**: Cada uno tiene un ID de 5 dígitos, edad, país, cuántos seguidores tiene, y su posición en las tendencias. Cada artista tiene su catálogo de álbumes.
- **Álbumes**: Tienen nombre, ID de 2 dígitos, fecha de lanzamiento, duración total, hasta 4 géneros musicales (Pop, Rock, Jazz, etc.), sello discográfico, ruta de la portada (un archivo PNG), y una puntuación del 1 al 10.
- **Canciones**: Cada una tiene nombre, un ID único de 9 dígitos (que combina el ID del artista + álbum + número de canción), duración, dos rutas de audio (una para 128kbps y otra para 320kbps, ambas en formato .ogg), los créditos (quién produjo, compuso, tocó los instrumentos), y cuántas veces se ha reproducido.
- **Listas de favoritos**: Solo los usuarios premium pueden tener una. No se permiten canciones duplicadas, y pueden seguir las listas de otros usuarios.
- **Anuncios**: Hay hasta 50 anuncios diferentes, cada uno con un mensaje de máximo 500 caracteres. Hay tres tipos: C (prioridad normal), B (prioridad media), y AAA (prioridad alta, aparece 3 veces más que los tipo C). Para usuarios estándar, aparecen cada dos canciones de forma aleatoria y sin repetirse.

**¿Cómo se relacionan todas estas cosas?**
- Un artista **compone** álbumes (relación fuerte: si eliminas el artista, se eliminan sus álbumes)
- Un álbum **contiene** canciones (relación fuerte: si eliminas el álbum, se eliminan sus canciones)
- Un usuario premium **tiene** una lista de favoritos (relación fuerte)
- Un usuario puede **seguir** la lista de favoritos de otro usuario (relación débil)
- Las canciones **tienen** créditos (relación débil)

**¿Qué funcionalidades principales necesito?**
- **Cargar y guardar datos**: Todo debe persistir en archivos cuando cerramos el programa
- **Sistema de login**: Los usuarios pueden entrar con su nickname
- **Menús diferentes**: Según si eres estándar o premium, verás opciones diferentes
- **Reproducción de música**: Con temporizador real (usando chronos), límites de K=5 y M=6 para las búsquedas
- **Gestión de favoritos**: Agregar, quitar, seguir listas de otros usuarios
- **Métricas**: Contar iteraciones y memoria total usada

**¿Qué restricciones importantes tengo?**
- **No puedo usar STL**: Tengo que crear mis propias estructuras de datos (listas, árboles, etc.)
- **Memoria dinámica propia**: No usar vector, map, list, etc. Todo a mano
- **Sin duplicados**: No puede haber usuarios, canciones o álbumes repetidos
- **Rutas absolutas de Linux**: Todas las rutas deben ser completas
- **Validaciones**: No se puede ir "atrás" si estás al inicio, etc.

**¿Qué va a ser más complicado de implementar?**
- **Manejar memoria sin STL**: Tengo que crear contadores para saber exactamente cuánta memoria uso
- **Sistema de anuncios ponderado**: No es solo aleatorio, sino que algunos tipos aparecen más que otros
- **Temporizador real**: Usar chronos para simular que realmente se reproduce la música (3 segundos por canción)
- **Búsquedas eficientes**: Con hasta 10.000 canciones en favoritos, necesito que las búsquedas sean rápidas

### Consideraciones importantes antes de empezar a programar

**¿Cómo voy a hacer que sea eficiente?**
- Usar referencias en lugar de copias cuando sea posible (así no desperdicio memoria)
- Elegir estructuras de datos que sean rápidas para lo que más voy a hacer
- Crear mi propio sistema de hash para los IDs únicos, así las búsquedas son súper rápidas

**¿Cómo voy a aplicar bien los conceptos de POO?**
- **Abstracción**: Cada cosa importante (Usuario, Artista, Canción) será una clase
- **Encapsulación**: Los datos privados solo se pueden acceder a través de métodos públicos
- **Herencia**: Usuarios estándar y premium heredarán de una clase Usuario base
- **Sobrecarga**: Usar operadores como + para agregar canciones, << para imprimir
- **Plantillas**: Crear listas genéricas que funcionen para cualquier tipo de dato
- **Funciones amigas**: Para acceder a datos privados cuando sea necesario

**¿Qué riesgos debo tener en cuenta?**
- **Rutas con espacios**: En Linux hay que escapar los espacios en las rutas
- **Validación de IDs**: Los IDs tienen formatos específicos que hay que verificar
- **Fugas de memoria**: Si no libero la memoria correctamente, el programa se come toda la RAM

**¿Cómo voy a probar que funciona bien?**
- **Casos extremos:** ¿Qué pasa si una lista está vacía? ¿Si un usuario premium no tiene favoritos?
- **Probar con datos reales:** Cargar muchos usuarios, canciones, álbumes
- Verificar que no haya fugas de memoria después de usar el programa mucho tiempo

## 2. Consideraciones para la Alternativa de Solución Propuesta
 
**¿Cómo voy a organizar los datos?**

En vez de usar las estructuras automáticas de C++ (como las del STL), aquí voy a crear las mías. ¿Por qué? Porque eso me da control total sobre la memoria y el rendimiento, y además es lo que me solicita explícitamente el desafío. Así que la idea es la siguiente:
 
- **Listas enlazadas propias:** Si quiero una lista de álbumes, canciones o favoritos, en vez de un `vector`, voy a hacer mi propia lista. Añadir o quitar algo es rapidísimo (en cualquier punto), aunque para buscar sí puede tardar un poco más si hay muchos elementos, pero para menos de 10 mil canciones o favoritos funciona bien.
- **Árbol binario de búsqueda:** Cuando quiero encontrar rápidamente a un usuario o artista por su ID o nickname, usaré un árbol. Así, buscar a alguien será mucho más rápido que revisar todo uno por uno.
- **Array dinámico propio para anuncios:** Como solo puede haber unos 50 anuncios, no necesito algo súper complejo. Un arreglo dinámico sencillo, diseñado a mano, me permite organizarlos y elegir cuál mostrar fácilmente y sin desperdiciar memoria.
- **Gestor central:** Todo estará coordinado por una clase "GestorUdeATunes", que guarda los punteros principales a todas estas listas y árboles. Esto me da un punto de control fácil para acceder a todo lo importante.
- **Guardar y cargar información:** Toda la información importante (usuarios, canciones, etc.) se guardará en archivos de texto separados, con un formato tipo CSV sencillo de leer y escribir a mano. Para esto es necesario dedicar una parte del código para convertir cada objeto en texto y viceversa.
 
- **¿Ventajas?** Muchísimo control sobre cómo uso la memoria y el tiempo que tarda cada operación. También, puedo contar exactamente cuánta memoria uso y cuántas vueltas doy en una búsqueda.

- **¿Y las desventajas?** Buscar por listas puede ser lento si hay demasiados elementos, por eso uso árboles para lo más crítico

## 3. ¿Cómo estará organizado el programa?
 
- **Clases base y herencia:** Empiezo creando una clase base (por ejemplo, para cualquier objeto con ID), y de ahí heredo para cosas más específicas, como usuarios o artistas. Así puedo aprovechar código y personalizar según el tipo.
- **Diferenciar usuarios estándar y premium:** Creo una clase "Usuario" general y de ahí derivo las versiones "Estándar" y "Premium". Esto me ayuda a mostrar menús y opciones diferentes según el tipo de usuario.
- **Dividir en módulos:** El código estará organizado en módulos bien definidos, como el menú general, el reproductor, o el editor de favoritos, para que cada cosa tenga su espacio y responsabilidad.
- **Medición de recursos:** El programa tendrá una especie de función "espía" que camina por todas mis estructuras y suma el tamaño que ocupan en memoria. Así sabré exactamente cuánto he usado. También, cada vez que recorra una estructura, llevaré una cuenta de cuántas repeticiones hago.
- **Uso de plantillas:** Cuando quiera listas genéricas (por ejemplo, para favoritos o para los créditos de una canción), usaré plantillas propias para no tener que programar lo mismo mil veces.
- **Sobrecargar operadores:** Por ejemplo, para agregar una canción fácilmente a una lista, usaré el operador `+`, y para acceder a cualquier elemento de una lista, el operador corchetes `[]`.
 
En resumen, quiero que el programa esté bien organizado, sea escalable para el futuro y, sobre todo, que sea fácil de entender y de mantener.

## 4. Diagrama de Clases - ¿Cómo se ven todas estas clases juntas?

Bueno, aquí está el "plano arquitectónico" de mi programa. Es como ver un mapa de cómo todas las clases se relacionan entre sí. Lo hice con Mermaid para que se vea bonito y sea fácil de entender.

```mermaid
classDiagram
    class GestorUdeATunes {
        +cargarDatos()
        +guardarDatos()
        +medirRecursos() int, long
        -contadorIteraciones int
        -contadorMemoria long
    }

    class Usuario {
        -nickname string
        -tipoMembresia enum~Estándar, Premium~
        -ciudad string
        -pais string
        -fechaInscripcion Date
        +login(string, string) bool
        +cambiarMembresia(enum)
        <<abstract>>
    }

    class Estándar : Usuario {
        +reproducirConAds()
    }

    class Premium : Usuario {
        -listaFavoritos ListaFavoritos*
        +agregarFavorito(Cancion*)
        +seguirLista(Premium*)
    }

    class Artista {
        -id int~5dig~
        -edad int
        -pais string
        -seguidores int
        -posicionTendencias int
        +agregarAlbum(Album*)
    }

    class Album {
        -nombre string
        -id int~2dig~
        -fechaLanzamiento Date
        -duracionTotal float
        -generos Genre[4]
        -sello string
        -rutaPortada string
        -puntuacion float~1-10~
        +agregarCancion(Cancion*)
    }

    class Cancion {
        -nombre string
        -id int~9dig~
        -duracion float
        -ruta128 string
        -ruta320 string
        -creditos ListaCredito*
        -reproducciones int
        +reproducir(int kbps) string
        +operator+~(Cancion&) void
    }

    class Credito {
        -nombre string
        -apellido string
        -codigoSAC string~10char~
        -categoria enum~Productor, Musico, Compositor~
    }

    class ListaFavoritos {
        -canciones ListaPlantilla~Cancion*~ ~max 10000~
        +agregar(Cancion*) bool~no dup~
        +quitar(int id) bool
        +reproducirAleatoria(bool orden) void
        +operator<<~(ostream&) void
    }

    class Anuncio {
        -mensaje string~<500char~
        -categoria enum~C, B, AAA~
        +seleccionarPonderado() Anuncio*
    }

    %% Relaciones
    GestorUdeATunes ||--o{ Usuario : gestiona
    GestorUdeATunes ||--o{ Artista : gestiona
    Usuario <|-- Estándar
    Usuario <|-- Premium
    Premium ||--|| ListaFavoritos : tiene
    Premium }o--o{ Premium : sigue
    Artista ||--o{ Album : compone
    Album ||--o{ Cancion : contiene
    Cancion ||--o{ Credito : tiene
    GestorUdeATunes ||--o{ Anuncio : gestiona

    enum Genre {
        Pop
        Rock
        Jazz
        Classica
        Electronica
        HipHop
        Reggae
        Blues
        Latina
    }
```

**¿Qué me dice este diagrama?**

- **GestorUdeATunes**: Es mi "jefe supremo". Maneja todo el sistema y lleva la cuenta de memoria e iteraciones.
- **Herencia clara**: Usuario es la clase base, y de ahí salen Estándar y Premium con sus propias características.
- **Relaciones bien definidas**: Las líneas muestran quién compone qué, quién tiene qué, y quién puede hacer qué.
- **Encapsulación**: Todo lo privado (con -) está protegido, y lo público (con +) es lo que otros pueden usar.

## 5. ¿Cómo se va a realizar todo esto? - Plan de Desarrollo

### Paso a paso: ¿Por dónde empiezo?

**Fase 1: Crear las estructuras base - Domingo 19**
- Primero voy a crear mis propias estructuras de datos: listas enlazadas, árboles binarios, y arrays dinámicos
- Implementar las clases básicas: Usuario, Artista, Álbum, Canción
- Crear el sistema de archivos para guardar y cargar datos

**Fase 2: Implementar la lógica principal - Lunes 20**
- Desarrollar el sistema de login y menús diferenciados
- Crear la funcionalidad de reproducción con temporizador
- Implementar el sistema de anuncios ponderado
- Desarrollar las listas de favoritos para usuarios premium

**Fase 3: Integración y pruebas - Martes 21**
- Conectar todas las partes del sistema
- Implementar la medición de memoria e iteraciones
- Hacer pruebas exhaustivas con datos reales
- Optimizar el rendimiento y corregir bugs

**Fase 4: Pulimiento final - Jueves 22**
- Refinar la interfaz de usuario
- Documentar el código completamente
- Preparar la sustentación y entrega

**Fase 5: Video expositivo - Miercoles 23**
- Preparación del guión
- Grabación y exposición del desarrollo del desafio
- Edición

### ¿Qué herramientas y técnicas voy a usar?

**Para el desarrollo:**
- **C++ puro**: Sin STL, todo hecho a mano para tener control total
- **Qt-Creator:** El desarrollo debe ser en el framework de Qt
- **Git**: Para control de versiones y trabajo en equipo

**Para las estructuras de datos:**
- **Listas enlazadas simples**: Para álbumes y canciones
- **Listas enlazadas dobles**: Para navegación bidireccional
- **Árboles binarios de búsqueda**: Para usuarios y artistas
- **Arrays dinámicos**: Para anuncios y géneros

**Para la persistencia:**
- **Archivos CSV**: Formato simple y legible
- **Serialización manual**: Convertir objetos a texto y viceversa
- **Rutas absolutas**: Para garantizar compatibilidad con Linux

### ¿Cómo voy a asegurar la calidad?

**Pruebas que haré:**
- **Pruebas unitarias**: Cada clase por separado
- **Pruebas de integración**: Cómo trabajan juntas las clases
- **Pruebas de estrés**: Con muchos datos para ver si aguanta
- **Pruebas de memoria**: Verificar que no haya fugas

**Métricas que voy a medir:**
- **Tiempo de respuesta**: Cuánto tarda en cargar, buscar, reproducir
- **Uso de memoria**: Exactamente cuántos bytes usa cada parte
- **Iteraciones**: Cuántas vueltas da cada algoritmo
- **Escalabilidad**: Cómo se comporta con 10,000 canciones

## 6. Conclusión - ¿Por qué este enfoque va a funcionar?

Después de todo este análisis, creo que tengo un plan sólido para crear UdeATunes. ¿Por qué estoy seguro de que va a funcionar?

**Porque he pensado en todo:**
- No solo sé QUÉ voy a hacer, sino también CÓMO y POR QUÉ
- He identificado los puntos difíciles desde el principio
- Tengo un plan de desarrollo realista y por etapas

**Porque priorizo lo importante:**
- **Eficiencia**: Mis estructuras de datos están pensadas para ser rápidas donde más lo necesito
- **Mantenibilidad**: El código estará bien organizado y documentado
- **Escalabilidad**: Si en el futuro quiero agregar más funciones, la base estará lista

**Porque aplico bien los conceptos de POO:**
- **Abstracción**: Cada clase representa algo real del mundo
- **Encapsulación**: Los datos están protegidos y solo se acceden correctamente
- **Herencia**: Aprovechamos código común entre usuarios estándar y premium
- **Polimorfismo**: El mismo método puede hacer cosas diferentes según el tipo de usuario
- **Plantillas**: Evitamos repetir código para estructuras similares

**Porque soy realista:**
- Sé que va a ser difícil, especialmente manejar memoria sin STL
- Pero también sé que es posible y que tengo las herramientas para lograrlo
- He planeado tiempo extra para debugging y optimización

**El resultado final será:**
Un sistema de streaming musical completo, eficiente, y bien diseñado que demuestra dominio de la programación orientada a objetos en C++. Un programa que no solo cumple los requisitos, sino que lo hace de manera elegante y profesional.

**Y lo más importante:** al final tendré un proyecto del que podré estar orgulloso, que realmente funciona como un Spotify en miniatura, y que demuestra que puedo crear software complejo desde cero.

Sin más que añadir, entonces doy por finalizado la entrega del 17 de Octubre

